In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
ft=KeyedVectors.load("/content/drive/MyDrive/FYP/IMDb/FastText/ft")

In [ ]:
import pandas as pd
import numpy as np
url=url="/content/gdrive/MyDrive/FYP/IMDb/Tf-Idf/IMDB-tfidf.csv"
df=pd.read_csv(url)
y=df["sentiment"]

In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [ ]:
import tensorflow as tf
maxlen=66
max_words=len(ft.wv.vocab.keys())
print(max_words)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

41479


In [ ]:
len(ft.wv.vocab.keys())

41479

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  
  if word in ft and i < max_words:
    embed_vector=ft[word]
    
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,LSTM,Dense,GRU
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

Epoch 1/5
586/586 [==============================] - 32s 39ms/step - loss: 0.5483 - accuracy: 0.7068
Epoch 2/5
586/586 [==============================] - 23s 39ms/step - loss: 0.3783 - accuracy: 0.8441
Epoch 3/5
586/586 [==============================] - 23s 39ms/step - loss: 0.3086 - accuracy: 0.8836
Epoch 4/5
586/586 [==============================] - 23s 39ms/step - loss: 0.2336 - accuracy: 0.9169
Epoch 5/5
586/586 [==============================] - 23s 39ms/step - loss: 0.1783 - accuracy: 0.9415


In [ ]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.85624
0.8561159705853578
0.8587308144727663
0.856866599403987
